In [13]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from operator import itemgetter
import os

%matplotlib inline 

def GraphGenerator_AutoLabel(rects, ax, location):
    """
    Helper function
    """
    horizontalAlign = 'center'
    if location == "up":
        verticalAlign='bottom'
    else:
        verticalAlign='top'

    for rect in rects:
        h = rect.get_height()
        label = "%.2f" % (h)
        if location == "up":
            h = h*1.0
        else:
            h = h*1.0
        ax.text(rect.get_x()+0.1, h, label,
                ha=horizontalAlign, va=verticalAlign)

def DrawBarChart(Inputs, Title, Ymax = None, Ymin = None):
    """
    Draw the bar chart
    """
    '''
    Prepare chart input
    '''
    NameList = []
    Speedup = []
    for item in Inputs:
        NameList.append(item[0])
        Speedup.append(item[1])

    x = NameList
    y = Speedup
    '''
    Draw bar chart
    '''
    plt.rc('font', size=2.8)
    fig = plt.figure(dpi=600)
    width = 0.25 # the width of the bars

    # Split into N bar chart
    splitCount = 3
    step = len(x) // splitCount
    patch_speedup = mpatches.Patch(color='green', label='Speedup By ABC')
    percentage = 0.33

    for i in range(splitCount):
        indexStart = i*step
        indexEnd = indexStart + int(percentage * len(x))

        x_sub = x[indexStart:indexEnd]
        y_sub = y[indexStart:indexEnd]
        #x_sub_order = [x for x in range(indexStart, indexEnd)]
        index = np.arange(len(x_sub))
        ax = fig.add_subplot(splitCount, 1 , i+1)

        rects = ax.bar(index, y_sub, width, color='green')
        # Do we need to limit the y axis?
        if Ymin == None and Ymax != None:
            plt.ylim(ymax=Ymax)
        elif Ymin != None and Ymax == None:
            plt.ylim(ymin=Ymin)
        elif Ymin != None and Ymax != None:
            plt.ylim(ymin=Ymin, ymax=Ymax)

        y_label_loc = "up"
        GraphGenerator_AutoLabel(rects, ax, y_label_loc)

        XLength = len(x)
        ax.set_ylabel('Execution Time From ABC Relative to Clang -O3: {}% - {}%'.format(
            round((indexStart/XLength)*100), round((indexEnd/XLength)*100)))
        
        ax.set_xticks(index)
        ax.set_xticklabels(x_sub, rotation=270)

    fig.legend( handles=[patch_speedup] )
    fig.suptitle(Title, fontweight='bold', fontsize=10, y=1.0)
    plt.tight_layout()
    plt.show()
    fig.savefig(Title + ".png", dpi=fig.dpi, bbox_inches='tight')

        
if __name__ == '__main__':
    ResultDict = {}
    # Normally, you don't need to comment out the following to process the raw data.
    # "BuildTimeAnalysis.json" is already processed.
    """
    # read raw data into dict
    loc = 'raw-data/BuildTimeAnalysis'
    for root, dirs, files in os.walk(loc):
        for File in files:
            key = File.split('-')[0]
            ResultDict[key] = {}
            with open(os.path.join(root, File), 'r') as f:
                for line in f:
                    line = line.strip()
                    # e.g. PassID:1, Time:37
                    if line.startswith('PassID'):
                        info = line.split(',')
                        PassID = int(info[0].split(':')[1])
                        time = int(info[1].split(':')[1])
                        if ResultDict[key].get(PassID) is None:
                            ResultDict[key][PassID] = 0
                        ResultDict[key][PassID] += time
                    # e.g. tensorflow-predict-time:2388 
                    elif line.startswith('tensorflow-predict-time'):
                        if ResultDict[key].get('TensorflowRuntime') is None:
                            ResultDict[key]['TensorflowRuntime'] = 0
                        ResultDict[key]['TensorflowRuntime'] += int(line.split(':')[1])
                    # e.g. all-runtime:98323
                    elif line.startswith('all-runtime'):
                        if ResultDict[key].get('TotalRuntime') is None:
                            ResultDict[key]['TotalRuntime'] = 0
                        ResultDict[key]['TotalRuntime'] += int(line.split(':')[1])
    with open('BuildTimeAnalysis.json', 'w') as js:                                                                        
        json.dump(ResultDict, js)
    """
    ResultDict = json.load(open("BuildTimeAnalysis.json"))
    print(ResultDict)
    """
    ABC_cycles_mean = json.load(open("ABC_cycles_mean.json"))
    Orig_cycles_mean = json.load(open("Orig_cycles_mean.json"))
    Orig_cycles_sigma = json.load(open("Orig_cycles_sigma.json"))
    # Due to not all the results are available(some may be -1, which indicating failure)
    AvailableKeys = []
    for target, cycles in ABC_cycles_mean.items():
        if cycles != -1:
            AvailableKeys.append(target)
    SpeedupDict = {}
    '''
    SpeedupDict contains all the information that we need =)
    '''
    TotalSpeedup = 0.0
    for target in AvailableKeys:
        new = ABC_cycles_mean[target]
        old = Orig_cycles_mean[target]
        speedup = (old - new) / old
        sigma = Orig_cycles_sigma[target]
        SpeedupDict[target] = {}
        SpeedupDict[target]['new'] = new
        SpeedupDict[target]['old'] = old
        SpeedupDict[target]['speedup'] = speedup
        SpeedupDict[target]['sigma'] = sigma
        TotalSpeedup += speedup
    AvgSpeedup = TotalSpeedup / len(AvailableKeys)
    '''
    Discard the speedup or slow-down less than twice sigma
    '''
    SelectedSpeedupDict = SpeedupDict.copy()
    for target, info in SpeedupDict.items():
        if abs(info['new'] - info['old']) < 2*info['sigma']:
            SelectedSpeedupDict.pop(target, None)
    print("Number of programs that are our targets: {}".format(len(Orig_cycles_mean)))
    print("Number of programs that built successfully by ABC: {}".format(len(SpeedupDict)))
    print("Number of programs that speedup by ABC(larger than twice sigma): {}".format(len(SelectedSpeedupDict)))
    print("Average speedup of all the ABC built programs is: {}".format(AvgSpeedup))
    print("-------------------------------------------------------------------------------------")
    
    GraphData = []
    for target, info in SelectedSpeedupDict.items():
        GraphData.append([target, info['speedup']+1] )
    '''
    Sort the data in descending order for plotting graph.
    '''
    GraphData.sort(key=itemgetter(1), reverse=True)
    DrawBarChart(Inputs=GraphData, Title="Execution Time From ABC Relative to Clang 5.0.1", Ymax = GraphData[0][1]+0.10, Ymin = 0.5)
    print(GraphData)
    """
        
        
        


{'aha': {'1': 5516, '2': 112730, '3': 13403, '4': 3375, '5': 6253, '6': 11714, '7': 4844, '8': 30304, '9': 33921, '10': 21728, '11': 62108, '12': 9698, '13': 14114, '14': 29595, '15': 5232, '16': 19833, '17': 24224, '18': 66689, '19': 4541, '20': 9998, '21': 15851, '22': 2791, '23': 4472, '24': 10279, '25': 48635, '26': 68345, '27': 147226, '28': 24689, '29': 352720, '30': 127275, '31': 13198, '32': 27265, '33': 10126, '34': 11374, 'TensorflowRuntime': 453810, 'TotalRuntime': 2232695}, 'kc': {'1': 933960, '2': 45512659, '3': 8878831, '4': 1100047, '5': 1831773, '6': 4296063, '7': 965875, '8': 11088144, '9': 16349643, '10': 6702961, '11': 28432802, '12': 3890312, '13': 3589300, '14': 4743229, '15': 1450868, '16': 8081868, '17': 9815847, '18': 27076646, '19': 1108259, '20': 4049838, '21': 6578828, '22': 708206, '23': 857078, '24': 4368098, '25': 22675546, '26': 6873888, '27': 52240397, '28': 10718688, '29': 133134050, '30': 29115486, '31': 5401442, '32': 9429811, '33': 4139636, '34': 454